In [ ]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
import networkx as nx
import requests
from bs4 import BeautifulSoup

In [ ]:
# Load data
data = pandas.read_csv('publications.csv')

In [ ]:
# Parse unique authors
import ast 
authors = data['authors']
pubs = [x for x in data['title']]
authors = [ast.literal_eval(x) for x in authors.values]
unique = set()
for author_list in authors:
    for author in author_list:
        unique.add(author)
print("Found", len(unique), 'unique authors')
print("Found", len(pubs), 'unique pubs')

In [ ]:
import os
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
base = "https://scholar.google.com"
author_urls = []
for index, pub in enumerate(pubs):
    print('Fetching pub', index)
    url = f'https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&field1=Title&text1={pub}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    for entry in soup.find_all('div', attrs={"class": 'issue-item__content'}):
        try:
            title = entry.find('h5', attrs={'class': 'issue-item__title'})
            authors = entry.find('ul', attrs={'class': 'rlist--inline'})
            authors = authors.find_all('a')
            author_urls.append({'title': title.text, 'authors': [(author.text, author.get('href')) for author in authors]})
            break
        except Exception as e:
            print(e)
print(author_urls)

In [ ]:
d = pandas.DataFrame(author_urls)
print(d)
d.to_csv('authors.csv')